In [1]:
import csv

In [4]:
CSV_FILENAME = "../gitignore/asymptote/medium_HF_2023_Aug_19_14_24_04_.csv"

In [16]:
with open(CSV_FILENAME, newline='') as f:
    dr = csv.DictReader(f)
    data = [d for d in dr]
    tests = [fn for fn in dr.fieldnames if 'filename' != fn]
data

[{'filename': 'gitignore/asymptote//medium_HF_2023_Aug_19_14_24_04_1.cho',
  'test_100_359_89_p': '0.8245571551809627',
  'test_100_461_115_p': '0.0002673397780722273',
  'test_100_615_153_p': '0.062460526017962884',
  'test_100_768_192_p': '0.009203432801149167',
  'test_100_973_243_p': '0.0015608119860057894',
  'test_100_1229_307_p': '1.4118098740382233e-05',
  'test_100_1536_384_p': None,
  'test_100_1946_486_p': None,
  'test_100_2458_614_p': None,
  'test_100_3072_768_p': None,
  'test_100_3840_960_p': None,
  'test_100_4813_1203_p': None,
  'test_100_6042_1510_p': None,
  'test_100_7578_1894_p': None},
 {'filename': 'gitignore/asymptote//medium_HF_2023_Aug_19_14_24_04_2.cho',
  'test_100_359_89_p': '0.03589347108697702',
  'test_100_461_115_p': '0.8757877119644033',
  'test_100_615_153_p': '0.0009850342103576099',
  'test_100_768_192_p': '0.32580893402105354',
  'test_100_973_243_p': '0.1046324793875946',
  'test_100_1229_307_p': '0.022748323540832786',
  'test_100_1536_384_p': 

In [7]:
tests

['test_100_359_89_p',
 'test_100_461_115_p',
 'test_100_615_153_p',
 'test_100_768_192_p',
 'test_100_973_243_p',
 'test_100_1229_307_p',
 'test_100_1536_384_p',
 'test_100_1946_486_p',
 'test_100_2458_614_p',
 'test_100_3072_768_p',
 'test_100_3840_960_p',
 'test_100_4813_1203_p',
 'test_100_6042_1510_p',
 'test_100_7578_1894_p']

In [27]:
buckets = [
    {"sec" : frozenset(d['filename'] for d in data if d[t] is not None and float(d[t]) > 0.05),
     "ins" : frozenset(d['filename'] for d in data if d[t] is not None and float(d[t]) <= 0.05 and float(d[t]) > (0.05 ** 3)),
     "neg" : frozenset(d['filename'] for d in data if d[t] is not None and float(d[t]) <= (0.05 ** 3)),
     "no_test"  : frozenset(d['filename'] for d in data if d[t] is None)}
    for t in tests
]
buckets

[{'sec': frozenset({'gitignore/asymptote//medium_HF_2023_Aug_19_14_24_04_1.cho',
             'gitignore/asymptote//medium_HF_2023_Aug_19_14_24_04_10.cho',
             'gitignore/asymptote//medium_HF_2023_Aug_19_14_24_04_11.cho',
             'gitignore/asymptote//medium_HF_2023_Aug_19_14_24_04_12.cho',
             'gitignore/asymptote//medium_HF_2023_Aug_19_14_24_04_13.cho',
             'gitignore/asymptote//medium_HF_2023_Aug_19_14_24_04_14.cho',
             'gitignore/asymptote//medium_HF_2023_Aug_19_14_24_04_15.cho',
             'gitignore/asymptote//medium_HF_2023_Aug_19_14_24_04_16.cho',
             'gitignore/asymptote//medium_HF_2023_Aug_19_14_24_04_19.cho',
             'gitignore/asymptote//medium_HF_2023_Aug_19_14_24_04_20.cho',
             'gitignore/asymptote//medium_HF_2023_Aug_19_14_24_04_21.cho',
             'gitignore/asymptote//medium_HF_2023_Aug_19_14_24_04_22.cho',
             'gitignore/asymptote//medium_HF_2023_Aug_19_14_24_04_23.cho',
             'gitig

In [37]:
def sanky_name(test, state):
    if state in ("nt", "no_test", "generated"):
        return state
    else:
        return test.rstrip("_p") + "_" + state
    
def sn_weight(name):
    if "generated" == name:
        return 15
    elif name.endswith("sec"):
        return 7
    elif name.endswith("ins"):
        return 3
    elif name.endswith("neg"):
        return 1
    elif "no_test" == name:
        return 0
    else:
        raise Exception(name)

In [38]:
sanky_ast = [
    ("generated", len(buckets[0]["sec"]), sanky_name(tests[0], "sec")),
    ("generated", len(buckets[0]["ins"]), sanky_name(tests[0], "ins")),
    ("generated", len(buckets[0]["no_test"]), sanky_name(tests[0], "no_test"))
]

for (bl, tl), (bh, th) in zip(zip(buckets, tests), list(zip(buckets, tests))[1:]):
    for sl in ("sec", "ins", "neg", "no_test"):
        for sh in ("sec", "ins", "neg", "no_test"):
            nl = sanky_name(tl, sl)
            nh = sanky_name(th, sh)
            flow = len(bl[sl] & bh[sh])
            if nl != nh:
                sanky_ast.append((nl, flow, nh))
                
sanky_ast = sorted(filter(lambda tp: 0 != tp[1], sanky_ast),
                   key=lambda tp: sn_weight(tp[0]) + sn_weight(tp[2]),
                   reverse=True)

sanky_ast

[('generated', 25, 'test_100_359_89_sec'),
 ('generated', 9, 'test_100_359_89_ins'),
 ('test_100_359_89_sec', 16, 'test_100_461_115_sec'),
 ('test_100_461_115_sec', 13, 'test_100_615_153_sec'),
 ('test_100_615_153_sec', 7, 'test_100_768_192_sec'),
 ('test_100_768_192_sec', 7, 'test_100_973_243_sec'),
 ('test_100_973_243_sec', 6, 'test_100_1229_307_sec'),
 ('test_100_1229_307_sec', 5, 'test_100_1536_384_sec'),
 ('test_100_1536_384_sec', 2, 'test_100_1946_486_sec'),
 ('test_100_1946_486_sec', 2, 'test_100_2458_614_sec'),
 ('test_100_2458_614_sec', 2, 'test_100_3072_768_sec'),
 ('test_100_3072_768_sec', 1, 'test_100_3840_960_sec'),
 ('test_100_3840_960_sec', 1, 'test_100_4813_1203_sec'),
 ('test_100_4813_1203_sec', 1, 'test_100_6042_1510_sec'),
 ('test_100_6042_1510_sec', 1, 'test_100_7578_1894_sec'),
 ('test_100_359_89_sec', 9, 'test_100_461_115_ins'),
 ('test_100_359_89_ins', 6, 'test_100_461_115_sec'),
 ('test_100_461_115_sec', 8, 'test_100_615_153_ins'),
 ('test_100_461_115_ins', 2, '

In [39]:
for nl, flow, nh in sanky_ast:
    print(f"{nl}[{flow}]{nh}")

generated[25]test_100_359_89_sec
generated[9]test_100_359_89_ins
test_100_359_89_sec[16]test_100_461_115_sec
test_100_461_115_sec[13]test_100_615_153_sec
test_100_615_153_sec[7]test_100_768_192_sec
test_100_768_192_sec[7]test_100_973_243_sec
test_100_973_243_sec[6]test_100_1229_307_sec
test_100_1229_307_sec[5]test_100_1536_384_sec
test_100_1536_384_sec[2]test_100_1946_486_sec
test_100_1946_486_sec[2]test_100_2458_614_sec
test_100_2458_614_sec[2]test_100_3072_768_sec
test_100_3072_768_sec[1]test_100_3840_960_sec
test_100_3840_960_sec[1]test_100_4813_1203_sec
test_100_4813_1203_sec[1]test_100_6042_1510_sec
test_100_6042_1510_sec[1]test_100_7578_1894_sec
test_100_359_89_sec[9]test_100_461_115_ins
test_100_359_89_ins[6]test_100_461_115_sec
test_100_461_115_sec[8]test_100_615_153_ins
test_100_461_115_ins[2]test_100_615_153_sec
test_100_615_153_sec[7]test_100_768_192_ins
test_100_615_153_ins[4]test_100_768_192_sec
test_100_768_192_sec[4]test_100_973_243_ins
test_100_768_192_ins[3]test_100_97